In [2]:
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import torch

import csv

# sentences

In [3]:
questions = [
    "Что такое курс рубля и почему он меняется?",
    "Зачем Банк России перешел к режиму плавающего курса?",
    "Проводит ли Банк России операции с целью укрепить или ослабить рубль?"
]

answers = [
    """
    Курс рубля к иностранной валюте - это стоимость одного рубля, 
    выраженная в единицах иностранной валюты.
    Курс рубля меняется под влиянием спроса и предложения на 
    иностранную валюту на валютном рынке.

    На курс рубля могут влиять:
    - Объемы внешней торговли
    - Импортные и экспортные цены
    - Уровни инфляции и процентных ставок
    - Темпы экономического роста
    - Периоды нестабильности
    - Изменения денежно-кредитной политики
    """,
    """
    Банк России перешел к режиму плавающего курса в 2014 году.
    Плавающий курс позволяет:
    - Проводить самостоятельную денежно-кредитную политику
    - Снижать инфляцию
    - Экономике подстраиваться под меняющиеся внешние условия
    """,
    """
    В обычных условиях Банк России не вмешивается в курс рубля.
    Банк России может проводить операции на валютном рынке:
    - Для поддержания финансовой стабильности
    - Для пополнения международных резервов
    - В 2014 году Банк России проводил интервенции для поддержки рубля
    """
]


links = [
    "https://www.cbr.ru/about_br/publ/ddkp/",
    "https://www.cbr.ru/dkp/exchange_rate/",
    "https://cbr.ru/oper_br/o_a/currency/"
]


with open("test.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["answer", "link"])
    for answer, link in zip(answers, links):
        writer.writerow([answer, link])

In [4]:
import pandas as pd
test_data= pd.read_csv("test.csv") 
test_data["answer"] = test_data["answer"].str.replace("\n", "")

In [5]:
test_data['answer'][0]

'    Курс рубля к иностранной валюте - это стоимость одного рубля,     выраженная в единицах иностранной валюты.    Курс рубля меняется под влиянием спроса и предложения на     иностранную валюту на валютном рынке.    На курс рубля могут влиять:    - Объемы внешней торговли    - Импортные и экспортные цены    - Уровни инфляции и процентных ставок    - Темпы экономического роста    - Периоды нестабильности    - Изменения денежно-кредитной политики    '

In [6]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = model.to('cuda')
embeddings = model.encode(test_data['answer'])
print(embeddings)

[2024-03-13 00:06:12,072] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2024-03-13 00:06:12.585590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[[ 0.01131858 -0.16646416 -0.01336718 ... -0.02108256  0.02286252
   0.03454603]
 [ 0.03172838 -0.17987302 -0.0171023  ...  0.01832076 -0.07802641
   0.04500178]
 [ 0.1076953  -0.19871438 -0.01545429 ...  0.04401986 -0.05108446
   0.04408288]]


In [7]:
embeddings.shape

(3, 768)

In [8]:
# text_vectors = np.vstack(embeddings)
# text_vectors

# Понижение размерности с помощью PCA, пока до 3 компонентов на тест, лучше до ~300, когда будет большая база

In [9]:
pca = Pipeline(steps=[
    ('mean', StandardScaler(with_mean=True, with_std=False)),
    ('pca', PCA(n_components=3, random_state=42)),
    ('std', StandardScaler(with_mean=True, with_std=True))
])
pca.fit(embeddings)

Pipeline(steps=[('mean', StandardScaler(with_std=False)),
                ('pca', PCA(n_components=3, random_state=42)),
                ('std', StandardScaler())])

In [10]:
text_vectors = pca.transform(embeddings)
text_vectors.shape

(3, 3)

In [11]:
text_vectors

array([[ 1.4124196e+00, -7.1208276e-02, -1.2503168e+06],
       [-6.4454156e-01,  1.2587960e+00, -7.6895030e+06],
       [-7.6787800e-01, -1.1875869e+00, -6.6098790e+06]], dtype=float32)

# ClickHouse

In [10]:
# !pip install clickhouse_driver

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [11]:
# !pip install clickhouse_connect

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [20]:
# Создание таблицы
import clickhouse_driver

client = clickhouse_driver.Client(host='localhost')
client.execute("""
CREATE TABLE embeddings (
    id UInt32,
    embedding Array(Float32),
    link String
)
ENGINE = MergeTree
PRIMARY KEY (id)
""")

[]

In [ ]:
client.execute('SHOW TABLES')

In [13]:
#Заполнение таблицы
import clickhouse_connect

client = clickhouse_connect.get_client()

embeddings = text_vectors.tolist()
links = test_data['link'].tolist()
ids = list(range(len(text_vectors)))

data = []

for i in range(len(ids)):
    row = [ids[i], embeddings[i], links[i]]
    data.append(row)

client.insert('embeddings', data, column_names=['id', 'embedding', 'link'])

In [14]:
#Вывод
import clickhouse_driver

client = clickhouse_driver.Client(host='localhost')
client.execute("SELECT * FROM embeddings")

[(0,
  [1.4124195575714111, -0.07120827585458755, -1250316.75],
  'https://www.cbr.ru/about_br/publ/ddkp/'),
 (1,
  [-0.6445415616035461, 1.258795976638794, -7689503.0],
  'https://www.cbr.ru/dkp/exchange_rate/'),
 (2,
  [-0.767877995967865, -1.1875869035720825, -6609879.0],
  'https://cbr.ru/oper_br/o_a/currency/')]

In [15]:
result = client.execute("SELECT * FROM embeddings")
result_df = pd.DataFrame(result)

result_df

,0,1,2
0,0,"[1.4124195575714111, -0.07120827585458755, -12...",https://www.cbr.ru/about_br/publ/ddkp/
1,1,"[-0.6445415616035461, 1.258795976638794, -7689...",https://www.cbr.ru/dkp/exchange_rate/
2,2,"[-0.767877995967865, -1.1875869035720825, -660...",https://cbr.ru/oper_br/o_a/currency/


# Загружаем языковую модель и токенизатор

In [28]:
llm_dirname = '../nsu-ai/team_code/models/llm'


In [29]:
llm_model = AutoModelForCausalLM.from_pretrained(llm_dirname, torch_dtype=torch.float16, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(llm_dirname)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Question

In [74]:
sentence = [questions[1]]
sentence

['Зачем Банк России перешел к режиму плавающего курса?']

In [75]:
embedding = model.encode(sentence)
pca_embedding = pca.transform(embedding)
pca_embedding = np.squeeze(pca_embedding)
pca_embedding.tolist()

[-0.8993228077888489, 0.4237446188926697, -1691226603520.0]

# L2Distance closest embedding

In [76]:
query_vector = pca_embedding.tolist()

results = client.execute("""
SELECT id, embedding, link
FROM embeddings
ORDER BY L2Distance(embedding, {}) ASC
LIMIT 1
""".format(query_vector))


for row in results:
    id, embedding, link = row
    print(f"ID: {id}, Link: {link}, Answer: {test_data['answer'][id]}")

ID: 1, Link: https://www.cbr.ru/dkp/exchange_rate/, Answer:     Банк России перешел к режиму плавающего курса в 2014 году.    Плавающий курс позволяет:    - Проводить самостоятельную денежно-кредитную политику    - Снижать инфляцию    - Экономике подстраиваться под меняющиеся внешние условия    


In [77]:
found_text = test_data['answer'][results[0][0]]

In [78]:
found_text

'    Банк России перешел к режиму плавающего курса в 2014 году.    Плавающий курс позволяет:    - Проводить самостоятельную денежно-кредитную политику    - Снижать инфляцию    - Экономике подстраиваться под меняющиеся внешние условия    '

# Собираем промпт, над ним надо ещё поработать

In [79]:
full_prompt = '<s>[INST] вы полезный помощник банка россии, который вежливо отвечает на вопросы. ты знаешь что: ' + found_text + ' Вопрос: ' + sentence[0] + ' Отвечайте только то, в чем уверены.' + ' [/INST]'

In [80]:
full_prompt

'<s>[INST] вы полезный помощник банка россии, который вежливо отвечает на вопросы. ты знаешь что:     Банк России перешел к режиму плавающего курса в 2014 году.    Плавающий курс позволяет:    - Проводить самостоятельную денежно-кредитную политику    - Снижать инфляцию    - Экономике подстраиваться под меняющиеся внешние условия     Вопрос: Зачем Банк России перешел к режиму плавающего курса? Отвечайте только то, в чем уверены. [/INST]'

# Генерируем ответ

In [81]:
model_input = tokenizer(full_prompt, return_tensors="pt").to("cuda")
input_prompt = tokenizer.batch_decode(torch.tensor(model_input['input_ids']), skip_special_tokens=True)[0]

with torch.no_grad():
    generated_text = tokenizer.decode(llm_model.generate(**model_input, max_new_tokens=1000)[0], skip_special_tokens=True)
    
answer = ' '.join(generated_text[len(input_prompt):].split()).strip()

/tmp/ipykernel_98/1589065902.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_prompt = tokenizer.batch_decode(torch.tensor(model_input['input_ids']), skip_special_tokens=True)[0]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [82]:
answer

'Банк России перешел к режиму плавающего курса в 2014 году с целью обеспечить экономическую стабильность и устойчивость в стране. Плавающий курс позволяет банку России проводить самостоятельную денежно-кредитную политику, снижать инфляцию, и экономике подстраиваться под меняющиеся внешние условия. Вероятно, банк России перешёл к плавающему курсу из-за необходимости в большей экономической свободе и независимости от внешних факторов, таких как международные кредиты и валютные интервенции. Плавающий курс также позволяет банку России лучше контролировать внутренние экономические процессы и реагировать на внешние условия, такие как изменения в мировых рынках и экономических условиях. В целом, переход к плавающему курсу был необходимым шагом для банка России, чтобы обеспечить экономическую стабильность и устойчивость в стране, и лучше контролировать внутренние экономические процессы.'

In [83]:
test_data['answer'][1]

'    Банк России перешел к режиму плавающего курса в 2014 году.    Плавающий курс позволяет:    - Проводить самостоятельную денежно-кредитную политику    - Снижать инфляцию    - Экономике подстраиваться под меняющиеся внешние условия    '